<a href="https://colab.research.google.com/github/jsbrittain/dengue-hackathon-25/blob/main/Preprocessing_climate_data_hackton_8dec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [232]:
pip install darts

In [233]:
import pandas as pd
import xarray as xr
import seaborn as sns
# from tqdm import tqdm
from scipy.stats import pearsonr
import numpy as np
from sklearn.metrics import r2_score
import os
from darts.models import TCNModel
import pandas as pd
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.missing_values import fill_missing_values
from darts.utils.likelihood_models import GaussianLikelihood, QuantileRegression
from darts.dataprocessing.transformers import Scaler, StaticCovariatesTransformer
from darts.models import TFTModel, NBEATSModel
# Assuming 'series' is your TimeSeries object
from darts import concatenate
from darts.utils.callbacks import TFMProgressBar
import shutil
import torch
from pytorch_lightning.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from darts.models import NBEATSModel
from darts.models import AutoARIMA
from darts.models import XGBModel
import warnings
#import optuna
# Suppress warnings
warnings.filterwarnings("ignore")
# 1. Suppress PyTorch Lightning Hardware Messages
import logging
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning.utilities.rank_zero").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning.accelerators.cuda").setLevel(logging.WARNING)
logging.getLogger("darts").setLevel(logging.WARNING)
# =============================================================================
# 1. HELPER: Date Alignment
# =============================================================================
def to_month_end(date_series):
    """Aligns dates to the last day of the month."""
    return pd.to_datetime(date_series) + pd.offsets.MonthEnd(0)

In [234]:
#Small island focuses on Barbados, Grenada, St Barthelemy, St Martin, Dominican Republic and Puerto Rico


name="DOM" #DOM is dominican republic,
#name = "GRD" #Graneda
name = "PRI" # Puerto Rico
name = "BRB" #Barbados
name = "BLM" #St Barthelemy
name = "MAF" # San Martin

In [235]:
def to_month_end(date_series):
    """Aligns dates to the last day of the month."""
    return pd.to_datetime(date_series) + pd.offsets.MonthEnd(0)

In [236]:

#Loading part of climatic data (standardized precipitation index and standardized evapotranporation index) in a file
#IMP
for i in ["01","03","06","12"]:
  spi=xr.open_dataset("%s-spa%s.zs.nc"%(name,i))
  spi=spi.where(spi.time.dt.day==1,drop=True).to_dataframe()#the first day of the month contains information of the last 30 days in SPI 1 and 3
  #For spi and spie 6 it only has 1 data per month so this line is harmeless

  spie=xr.open_dataset("%s-spe%s.zs.nc"%(name,i))
  spie=spie.where(spie.time.dt.day==1,drop=True).to_dataframe()#the first day of the month contains information of the last 30 days in SPI 1 and 3
  if i=="01":
   inter_data=spi
   inter_data["spe%s" %i]=spie
  else:
       inter_data["spa%s" %i]=spi
       inter_data["spe%s" %i]=spie
inter_data.drop(index=inter_data.index[-1],inplace=True)
inter_data=inter_data.reset_index()
inter_data["time"]=inter_data["time"].dt.to_period("M")
inter_data

,time,region,spa01,spe01,spa03,spe03,spa06,spe06,spa12,spe12
0,2015-01,MAF,-0.478032,-0.495204,0.685821,0.884500,0.560449,0.655112,0.205576,0.301073
1,2015-02,MAF,-0.432038,-0.401639,-0.383223,-0.286572,0.226670,0.325741,0.163174,0.259818
2,2015-03,MAF,0.161315,0.484753,-0.548979,-0.471571,0.283702,0.429472,0.163755,0.282956
3,2015-04,MAF,-0.446388,-0.358705,-0.607785,-0.502383,0.070698,0.288033,0.099938,0.240527
4,2015-05,MAF,-1.402895,-1.510857,-1.193630,-1.301146,-1.222066,-1.238098,-0.193628,-0.164890
...,...,...,...,...,...,...,...,...,...,...
125,2025-06,MAF,0.274276,0.508309,0.867797,0.851866,0.402929,0.350832,0.733468,0.651406
126,2025-07,MAF,-1.325156,-1.605539,0.336730,0.402608,0.044114,-0.008117,0.483911,0.402579
127,2025-08,MAF,-0.896502,-1.113764,-0.706014,-0.793419,0.082582,0.147804,0.307838,0.271319
128,2025-09,MAF,-0.043789,0.197317,-0.799832,-0.821664,0.115409,0.210896,0.303621,0.296440


In [237]:
# Load of observation data (reanalysis)
reanalysis=xr.open_dataset('%s-reanalysis_monthly.zs.nc'%name)#.sel(time=slice("2014-12-31","2022-12-31")).to_dataframe()

reanalysis=reanalysis.to_dataframe().reset_index()
reanalysis["time"]=pd.to_datetime(reanalysis["time"]).dt.to_period("M")
reanalysis



,time,region,t2m,rh,tp
0,2015-01,MAF,298.422095,0.799222,0.479093
1,2015-02,MAF,298.380370,0.790267,0.492298
2,2015-03,MAF,298.117486,0.741720,0.330724
3,2015-04,MAF,298.431327,0.786269,0.399398
4,2015-05,MAF,299.337863,0.784502,0.449290
...,...,...,...,...,...
125,2025-06,MAF,300.204616,0.789534,0.207752
126,2025-07,MAF,300.467978,0.816346,0.432789
127,2025-08,MAF,301.137230,0.802202,0.956340
128,2025-09,MAF,301.131512,0.801790,1.464670


In [238]:

#load of climatic data
reanalysis=xr.open_dataset('%s-reanalysis_monthly.zs.nc'%name)#.sel(time=slice("2014-12-31","2022-12-31")).to_dataframe()

reanalysis=reanalysis.to_dataframe().reset_index()
reanalysis["time"]=pd.to_datetime(reanalysis["time"]).dt.to_period("M")
reanalysis


#Load of forecast data and group it into one dataset , forecast data will be classified as variable_LX, being LX the forecast week
# For examplke T2m_L2 is the forecast of temperature for the 2nd week when the forecast starts in "time"
#Load of reanlysis
forecast=xr.open_dataset("%s-seasonal_forecast_monthly.zs.nc"%name)

def unroll_months(ds: xr.Dataset) -> pd.DataFrame:
    if "month" not in ds.dims:
        raise ValueError("Only use this on seasonal_forecast_monthly data")
    months = ds['month'].values
    orig_vars = list(ds.data_vars)
    # create new month-sliced variables var_L1 ... var_L6 (each will be dims ('time','region'))
    for var in ds.data_vars:
        for m in months:
            # cast m to int if it's numeric for nicer names; use str(m) if not
            name = f"{var}_L{m}"
            ds[name] = ds[var].sel(month=m)

    ds = ds.drop_vars(orig_vars)
    if 'month' in ds.dims:
        ds = ds.drop_dims('month')
    return ds.to_dataframe().reset_index()
forecast=unroll_months(forecast)
forecast=forecast.reset_index()
forecast["time"]=pd.to_datetime(forecast["time"]).dt.to_period("M")
forecast

clim_data=pd.merge(reanalysis,forecast)
clim_data.columns.values

array(['time', 'region', 't2m', 'rh', 'tp', 'index', 't2m_L1.0',
       't2m_L2.0', 't2m_L3.0', 't2m_L4.0', 't2m_L5.0', 't2m_L6.0',
       'rh_L1.0', 'rh_L2.0', 'rh_L3.0', 'rh_L4.0', 'rh_L5.0', 'rh_L6.0',
       'tp_L1.0', 'tp_L2.0', 'tp_L3.0', 'tp_L4.0', 'tp_L5.0', 'tp_L6.0',
       'mx2t24_L1.0', 'mx2t24_L2.0', 'mx2t24_L3.0', 'mx2t24_L4.0',
       'mx2t24_L5.0', 'mx2t24_L6.0', 'mn2t24_L1.0', 'mn2t24_L2.0',
       'mn2t24_L3.0', 'mn2t24_L4.0', 'mn2t24_L5.0', 'mn2t24_L6.0'],
      dtype=object)

In [239]:
#including el nino index
nino=xr.open_dataset("elninossta.nc").to_dataframe().reset_index()
nino["time"]=pd.to_datetime(nino["time"]).dt.to_period("M")
nino


,time,ssta
0,2015-01,0.477419
1,2015-02,0.750000
2,2015-03,0.438710
3,2015-04,0.836667
4,2015-05,0.841935
...,...,...
126,2025-07,-0.083871
127,2025-08,-0.338710
128,2025-09,-0.423333
129,2025-10,-0.509677


In [240]:


clim_data=pd.merge(reanalysis,forecast)
clim_data.columns.values

array(['time', 'region', 't2m', 'rh', 'tp', 'index', 't2m_L1.0',
       't2m_L2.0', 't2m_L3.0', 't2m_L4.0', 't2m_L5.0', 't2m_L6.0',
       'rh_L1.0', 'rh_L2.0', 'rh_L3.0', 'rh_L4.0', 'rh_L5.0', 'rh_L6.0',
       'tp_L1.0', 'tp_L2.0', 'tp_L3.0', 'tp_L4.0', 'tp_L5.0', 'tp_L6.0',
       'mx2t24_L1.0', 'mx2t24_L2.0', 'mx2t24_L3.0', 'mx2t24_L4.0',
       'mx2t24_L5.0', 'mx2t24_L6.0', 'mn2t24_L1.0', 'mn2t24_L2.0',
       'mn2t24_L3.0', 'mn2t24_L4.0', 'mn2t24_L5.0', 'mn2t24_L6.0'],
      dtype=object)

In [241]:
clim_data

,time,region,t2m,rh,tp,index,t2m_L1.0,t2m_L2.0,t2m_L3.0,t2m_L4.0,...,mx2t24_L3.0,mx2t24_L4.0,mx2t24_L5.0,mx2t24_L6.0,mn2t24_L1.0,mn2t24_L2.0,mn2t24_L3.0,mn2t24_L4.0,mn2t24_L5.0,mn2t24_L6.0
0,2015-01,MAF,298.422095,0.799222,0.479093,0,298.262156,297.848017,297.906435,298.359358,...,298.518549,298.956532,299.683120,300.469403,297.664159,297.216519,297.261403,297.722682,298.470064,299.307841
1,2015-02,MAF,298.380370,0.790267,0.492298,1,298.267972,298.127263,298.478727,299.216134,...,299.095175,299.791871,300.554431,300.935655,297.662458,297.467438,297.819615,298.584316,299.373230,299.664779
2,2015-03,MAF,298.117486,0.741720,0.330724,2,298.179530,298.565515,299.201525,299.983490,...,299.765938,300.526657,300.911248,301.163399,297.537324,297.943292,298.572149,299.384741,299.698598,299.875935
3,2015-04,MAF,298.431327,0.786269,0.399398,3,298.402724,299.211821,299.944133,300.312538,...,300.496594,300.873878,301.178561,301.386518,297.766761,298.577332,299.343368,299.699694,299.902642,299.980319
4,2015-05,MAF,299.337863,0.784502,0.449290,4,299.372437,300.065739,300.448553,300.755132,...,301.012002,301.356501,301.557900,301.421963,298.796631,299.481977,299.817579,300.061469,300.090239,300.057912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,2025-06,MAF,300.204616,0.789534,0.207752,125,300.273110,300.622800,300.911855,301.008476,...,301.546812,301.673730,301.433586,300.700013,299.585948,299.912212,300.144208,300.179972,299.923590,299.343758
126,2025-07,MAF,300.467978,0.816346,0.432789,126,300.515672,300.920872,301.028709,300.794755,...,301.689541,301.472795,300.735818,299.793019,299.772005,300.122999,300.199645,299.927534,299.345549,298.517356
127,2025-08,MAF,301.137230,0.802202,0.956340,127,301.019368,301.117225,300.838293,300.099064,...,301.530721,300.758979,299.852025,299.055957,300.163389,300.236202,299.941448,299.287528,298.577261,297.805822
128,2025-09,MAF,301.131512,0.801790,1.464670,128,301.109821,300.862369,300.194423,299.261439,...,300.854434,299.836624,299.000803,298.647412,300.203519,299.925898,299.385041,298.589269,297.726785,297.411262


In [242]:
clim_covs=pd.merge(inter_data,clim_data,on=["time","region"])
clim_covs
clim_covs= pd.merge(clim_covs,nino,on="time")#Adding el nino index (global index so we cannot join by region)
test=clim_covs
test

,time,region,spa01,spe01,spa03,spe03,spa06,spe06,spa12,spe12,...,mx2t24_L4.0,mx2t24_L5.0,mx2t24_L6.0,mn2t24_L1.0,mn2t24_L2.0,mn2t24_L3.0,mn2t24_L4.0,mn2t24_L5.0,mn2t24_L6.0,ssta
0,2015-01,MAF,-0.478032,-0.495204,0.685821,0.884500,0.560449,0.655112,0.205576,0.301073,...,298.956532,299.683120,300.469403,297.664159,297.216519,297.261403,297.722682,298.470064,299.307841,0.477419
1,2015-02,MAF,-0.432038,-0.401639,-0.383223,-0.286572,0.226670,0.325741,0.163174,0.259818,...,299.791871,300.554431,300.935655,297.662458,297.467438,297.819615,298.584316,299.373230,299.664779,0.750000
2,2015-03,MAF,0.161315,0.484753,-0.548979,-0.471571,0.283702,0.429472,0.163755,0.282956,...,300.526657,300.911248,301.163399,297.537324,297.943292,298.572149,299.384741,299.698598,299.875935,0.438710
3,2015-04,MAF,-0.446388,-0.358705,-0.607785,-0.502383,0.070698,0.288033,0.099938,0.240527,...,300.873878,301.178561,301.386518,297.766761,298.577332,299.343368,299.699694,299.902642,299.980319,0.836667
4,2015-05,MAF,-1.402895,-1.510857,-1.193630,-1.301146,-1.222066,-1.238098,-0.193628,-0.164890,...,301.356501,301.557900,301.421963,298.796631,299.481977,299.817579,300.061469,300.090239,300.057912,0.841935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,2025-06,MAF,0.274276,0.508309,0.867797,0.851866,0.402929,0.350832,0.733468,0.651406,...,301.673730,301.433586,300.700013,299.585948,299.912212,300.144208,300.179972,299.923590,299.343758,0.056667
126,2025-07,MAF,-1.325156,-1.605539,0.336730,0.402608,0.044114,-0.008117,0.483911,0.402579,...,301.472795,300.735818,299.793019,299.772005,300.122999,300.199645,299.927534,299.345549,298.517356,-0.083871
127,2025-08,MAF,-0.896502,-1.113764,-0.706014,-0.793419,0.082582,0.147804,0.307838,0.271319,...,300.758979,299.852025,299.055957,300.163389,300.236202,299.941448,299.287528,298.577261,297.805822,-0.338710
128,2025-09,MAF,-0.043789,0.197317,-0.799832,-0.821664,0.115409,0.210896,0.303621,0.296440,...,299.836624,299.000803,298.647412,300.203519,299.925898,299.385041,298.589269,297.726785,297.411262,-0.423333


In [243]:

#Load epidemology and population and gpd data if available data


ccvi_path = "%s-ccvi.zs.nc" % name

No_data=[]

if os.path.exists(ccvi_path):
 ccvi_ds = xr.open_dataset("%s-ccvi.zs.nc"%name)
 ccvi_df = ccvi_ds.to_dataframe().reset_index()
 ccvi_df['time'] = pd.to_datetime(ccvi_df['time'])
 ccvi_df = ccvi_df.set_index('time')
 ccvi_df_monthly = ccvi_df.resample('MS').ffill()
 ccvi_df_monthly = ccvi_df_monthly.reset_index()
 ccvi_df_monthly['time'] = pd.to_datetime(ccvi_df_monthly['time']).dt.to_period('M')
 test=pd.merge(test, ccvi_df_monthly,on=["time","region"])
else:
  No_data.append("ccvi")
gdp_path = "%s-gdp_pc.zs.nc" % name
if os.path.exists("%s-gdp_pc.zs.nc"%name):
 gdp_ds = xr.open_dataset("%s-gdp_pc.zs.nc"%name)
 gdp_df = gdp_ds.to_dataframe().reset_index()
 gdp_df['time'] = pd.to_datetime(gdp_df['time'])
 gdp_df = gdp_df.set_index('time')
 gdp_df_monthly = gdp_df.resample('MS').ffill()
 gdp_df_monthly = gdp_df_monthly.reset_index()
 gdp_df_monthly['time'] = pd.to_datetime(gdp_df_monthly['time']).dt.to_period('M')
 test=pd.merge(test,gdp_df_monthly,on=["time","region"])
else:
 No_data.append("gdp")

pop_path = "%s-population.zs.nc" % name
if os.path.exists("%s-population.zs.nc"%name):
 pop_ds = xr.open_dataset("%s-population.zs.nc"%name)
 pop_df = pop_ds.to_dataframe().reset_index()
 pop_df['time'] = pd.to_datetime(pop_df['time'])
 pop_df = pop_df.set_index('time')
 pop_df_monthly = pop_df.resample('MS').ffill()
 pop_df_monthly = pop_df_monthly.reset_index()
 # Convert time to Year-Month (Period[M])
 pop_df_monthly['time'] = pd.to_datetime(pop_df_monthly['time']).dt.to_period('M')
 test=pd.merge(test,pop_df_monthly,on=["time","region"])
else:
 No_data.append("population")

print("No data on" + str(No_data))

No data on['ccvi']


In [244]:
#Load of epidemological data
df_epi= pd.read_csv("%s_epi_training.csv"%name)

df_epi['Date'] = to_month_end(df_epi['Month'])
df_epi = df_epi.drop(columns=['Month', 'Unnamed: 0'], errors='ignore')
if df_epi.duplicated(subset=['Date']).any():
    df_epi = df_epi.groupby(['Date']).mean(numeric_only=True).reset_index()
df_epi

,Country,Region,Cases,Spatial_Resolution,Date
0,SAINT MARTIN,SAINT MARTIN,0,Admin0,2014-12-31
1,SAINT MARTIN,SAINT MARTIN,106,Admin0,2015-01-31
2,SAINT MARTIN,SAINT MARTIN,119,Admin0,2015-02-28
3,SAINT MARTIN,SAINT MARTIN,62,Admin0,2015-03-31
4,SAINT MARTIN,SAINT MARTIN,0,Admin0,2015-04-30
...,...,...,...,...,...
92,SAINT MARTIN,SAINT MARTIN,0,Admin0,2022-08-31
93,SAINT MARTIN,SAINT MARTIN,0,Admin0,2022-09-30
94,SAINT MARTIN,SAINT MARTIN,0,Admin0,2022-10-31
95,SAINT MARTIN,SAINT MARTIN,0,Admin0,2022-11-30


In [245]:
df_epi=df_epi.reset_index()
df_epi["time"]=pd.to_datetime(df_epi["Date"]).dt.to_period("M")
df_epi=df_epi.drop(columns="Date")
df_epi

,index,Country,Region,Cases,Spatial_Resolution,time
0,0,SAINT MARTIN,SAINT MARTIN,0,Admin0,2014-12
1,1,SAINT MARTIN,SAINT MARTIN,106,Admin0,2015-01
2,2,SAINT MARTIN,SAINT MARTIN,119,Admin0,2015-02
3,3,SAINT MARTIN,SAINT MARTIN,62,Admin0,2015-03
4,4,SAINT MARTIN,SAINT MARTIN,0,Admin0,2015-04
...,...,...,...,...,...,...
92,92,SAINT MARTIN,SAINT MARTIN,0,Admin0,2022-08
93,93,SAINT MARTIN,SAINT MARTIN,0,Admin0,2022-09
94,94,SAINT MARTIN,SAINT MARTIN,0,Admin0,2022-10
95,95,SAINT MARTIN,SAINT MARTIN,0,Admin0,2022-11


In [246]:
test=pd.merge(test,df_epi,on=["time"])
test

,time,region,spa01,spe01,spa03,spe03,spa06,spe06,spa12,spe12,...,ssta,gdp_pc,pop_count,area,pop_density,index_y,Country,Region,Cases,Spatial_Resolution
0,2015-01,MAF,-0.478032,-0.495204,0.685821,0.884500,0.560449,0.655112,0.205576,0.301073,...,0.477419,39979.590460,34361.819513,68.281634,503.236635,1,SAINT MARTIN,SAINT MARTIN,106,Admin0
1,2015-02,MAF,-0.432038,-0.401639,-0.383223,-0.286572,0.226670,0.325741,0.163174,0.259818,...,0.750000,39979.590460,34361.819513,68.281634,503.236635,2,SAINT MARTIN,SAINT MARTIN,119,Admin0
2,2015-03,MAF,0.161315,0.484753,-0.548979,-0.471571,0.283702,0.429472,0.163755,0.282956,...,0.438710,39979.590460,34361.819513,68.281634,503.236635,3,SAINT MARTIN,SAINT MARTIN,62,Admin0
3,2015-04,MAF,-0.446388,-0.358705,-0.607785,-0.502383,0.070698,0.288033,0.099938,0.240527,...,0.836667,39979.590460,34361.819513,68.281634,503.236635,4,SAINT MARTIN,SAINT MARTIN,0,Admin0
4,2015-05,MAF,-1.402895,-1.510857,-1.193630,-1.301146,-1.222066,-1.238098,-0.193628,-0.164890,...,0.841935,39979.590460,34361.819513,68.281634,503.236635,5,SAINT MARTIN,SAINT MARTIN,0,Admin0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,2021-09,MAF,-1.075683,-1.177828,-0.420942,-0.357429,-0.478441,-0.405144,-0.323752,-0.296940,...,-0.236667,33019.644911,27922.844537,68.281634,408.936387,81,SAINT MARTIN,SAINT MARTIN,0,Admin0
81,2021-10,MAF,-0.140812,0.071785,-0.436822,-0.320835,-0.560489,-0.568319,-0.478533,-0.462405,...,-0.761290,33019.644911,27922.844537,68.281634,408.936387,82,SAINT MARTIN,SAINT MARTIN,0,Admin0
82,2021-11,MAF,-0.880048,-0.936369,-1.007067,-1.034414,-0.670259,-0.714564,-1.014429,-1.090084,...,-0.890000,33019.644911,27922.844537,68.281634,408.936387,83,SAINT MARTIN,SAINT MARTIN,0,Admin0
83,2021-12,MAF,-0.815280,-1.266850,-1.088859,-1.242159,-1.073846,-1.133422,-1.004187,-1.087903,...,-1.032258,33019.644911,27922.844537,68.281634,408.936387,84,SAINT MARTIN,SAINT MARTIN,0,Admin0


In [247]:
test.columns.values

array(['time', 'region', 'spa01', 'spe01', 'spa03', 'spe03', 'spa06',
       'spe06', 'spa12', 'spe12', 't2m', 'rh', 'tp', 'index_x',
       't2m_L1.0', 't2m_L2.0', 't2m_L3.0', 't2m_L4.0', 't2m_L5.0',
       't2m_L6.0', 'rh_L1.0', 'rh_L2.0', 'rh_L3.0', 'rh_L4.0', 'rh_L5.0',
       'rh_L6.0', 'tp_L1.0', 'tp_L2.0', 'tp_L3.0', 'tp_L4.0', 'tp_L5.0',
       'tp_L6.0', 'mx2t24_L1.0', 'mx2t24_L2.0', 'mx2t24_L3.0',
       'mx2t24_L4.0', 'mx2t24_L5.0', 'mx2t24_L6.0', 'mn2t24_L1.0',
       'mn2t24_L2.0', 'mn2t24_L3.0', 'mn2t24_L4.0', 'mn2t24_L5.0',
       'mn2t24_L6.0', 'ssta', 'gdp_pc', 'pop_count', 'area',
       'pop_density', 'index_y', 'Country', 'Region', 'Cases',
       'Spatial_Resolution'], dtype=object)

In [248]:
test.to_csv("Covariates_data_%s.csv"%name)

In [249]:
pwd

'/content'